In [1]:
%matplotlib inline

import json
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from scipy import interpolate
from operator import itemgetter
import matplotlib as mpl
from astropy.stats import sigma_clip

plt.style.use('seaborn-white')
mpl.rcParams.update({'font.size': 16})

## Optimal Lowercut Method: Shifting the Most Precise Lowercut Measurement

The lowercut value appears to be key to TAME assigning a good equivalent width measurement for a given line. This notebook uses measurements taken from ~20 stars at 10 different lowercut values to find the best lowercut value per line. Prior to this method, we've been attempting to find the optimal lowercut value as the value where the typical (or average) percent difference with respect to our known values is minimized. We've found that the solution space for these measurements can be broad and certain solutions have large spread. Therefore, this method takes a different approach. We select the optimal lowercut value as the value where the spread in percent different is minimized and apply an offset to get the percent difference of that lowercut value centered on zero.

In [ ]:

outputs = glob.glob("tame-1.1.0/*14.json")
by_hand = glob.glob("tame-1.1.0/*.ew")
lc_range = np.arange(0.95,0.995,0.0001)
len_range = 10
#Fine Sampling
#outputs = glob.glob("tame-1.1.0/finesampling.json")
#by_hand = glob.glob("tame-1.1.0/col110_5125*.ew")


d = {'JSON File': outputs, 'By-Hand File': by_hand}
disp_df = pd.DataFrame(data=d)
disp_df